## copy sklearn `MLPClassifier` weights to torch or OpenSoundscape model

In [1]:
import sklearn.neural_network
import torch
from torch import nn
import numpy as np
import opensoundscape as opso

/Users/SML161/opensoundscape/opensoundscape/ml/cnn.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


say you created an sklearn model like this

In [2]:
hidden_layer_sizes = (100,100,100)
input_size = 512
output_size = 3
mlp = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=hidden_layer_sizes)
mlp.fit(torch.rand((100,input_size)), torch.randint(0, 2, (100,output_size)))

MLPClassifier(hidden_layer_sizes=(100, 100, 100))

define a class that is simply a set of sequential fully-connected layers with relu activation (ie a "multi-layer perceptron" MLP) - this is the same as the sklearn MLPCLassifier

In [3]:
class TorchMLP(nn.Module):
    def __init__(self, hidden_layer_sizes, input_size, output_size):
        super().__init__()        
        # Define the MLP layers
        # first layer maps input size : hidden layer size [0]
        # last layer maps hidden layer size[-1] : output size
        # inner layers map from one hidden layer size to the next
        all_layer_sizes = [input_size] + list(hidden_layer_sizes) + [output_size]
        self.layers = nn.ModuleList()
        for i in range(len(all_layer_sizes) - 1):
            self.layers.append(nn.Linear(all_layer_sizes[i], all_layer_sizes[i+1]))
            
    def forward(self, x):
        # Pass through the MLP layers and relu activation
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        # no relu on the last layer
        x = self.layers[-1](x)
        return x


In [4]:
# Initialize the shallow torch classifier with random weights and the correct layer sizes
shallow_classifier = TorchMLP(hidden_layer_sizes=hidden_layer_sizes, input_size=input_size, output_size=output_size)

# Extract weights and biases from the MLPClassifier
mlp_coefs = mlp.coefs_  # List of weight matrices
mlp_intercepts = mlp.intercepts_  # List of bias vectors

# Copy the weights and biases from the MLPClassifier to the modified model
with torch.no_grad():
    for i, (layer, (coef, intercept)) in enumerate(zip(shallow_classifier.layers, zip(mlp_coefs, mlp_intercepts))):
        layer.weight = nn.Parameter(torch.Tensor(coef).T) #transpose because layer with input x output y wants weights with shape [y, x]
        layer.bias = nn.Parameter(torch.Tensor(intercept))

# set to evaluation mode (not training)
shallow_classifier.eval()

TorchMLP(
  (layers): ModuleList(
    (0): Linear(in_features=512, out_features=100, bias=True)
    (1-2): 2 x Linear(in_features=100, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=3, bias=True)
  )
)

#### CHECK: our re-created shallow classifier and the original SKLearn MLPClassifier generate the same output for a given sample

In [5]:
sample = torch.rand((2,input_size))


In [6]:
torch.sigmoid(shallow_classifier(sample))

tensor([[7.8280e-01, 1.1035e-04, 3.3725e-02],
        [8.5589e-01, 3.0727e-02, 7.6130e-01]], grad_fn=<SigmoidBackward0>)

In [7]:
mlp.predict_proba(sample) #includes the sigmoid activation function

array([[7.82803272e-01, 1.10347810e-04, 3.37247157e-02],
       [8.55887551e-01, 3.07272804e-02, 7.61303803e-01]])

# use the classifier in an opso model

make sure that the dimensions will match: the embedding dimension of the network should equal the input to the shallow classifier

In [9]:
m = opso.ml.cnn.CNN('resnet18', classes=[0], sample_duration=2)
print(f'input size for this architectures classifier: {m.network.fc.weight.shape[1]}')
print(f'input size used above to create the MLPCLassifier: {input_size}')

input size for this architectures classifier: 512
input size used above to create the MLPCLassifier: 512


#### replace an opso.CNN's last fully connected layer with this shallow classifier

In [10]:
m.network.fc = shallow_classifier # for non-resnet architectures, the classifier might be called something other than .fc
m.classes = np.arange(output_size) # maybe you have informative class names, but maybe they are unknown

can now run it on audio files like any other opso classifier

In [11]:
m.predict('/Users/SML161/a.mp3')

  0%|          | 0/2 [00:00<?, ?it/s]

0         1          2
file                start_time end_time                                
/Users/SML161/a.mp3 0.0        2.0      -11.668139 -3.368650  -8.482996
                    2.0        4.0      -10.239831 -1.379559 -10.250764